# Stateless / statefull  Word count Example

## Import librairies 

In [ ]:

import findspark
findspark.init('C:\spark\spark-2.3.0-bin-hadoop2.7')
from pyspark.sql import SparkSession 
from pyspark import SparkContext, SparkConf
from pyspark.streaming.kafka import KafkaUtils
from pyspark.streaming import StreamingContext
import time
import os
import sys

## Set Paths 

In [ ]:
SCALA_VERSION = '2.11'
SPARK_VERSION = '2.3.0'
spark_path = r"C:\spark\spark-2.3.0-bin-hadoop2.7" # spark installed folder
os.environ['SPARK_HOME'] = spark_path
#os.environ['PYSPARK_SUBMIT_ARGS']="--master local[2] pyspark-shell"
os.environ['PYSPARK_SUBMIT_ARGS']='--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.3.0 pyspark-shell'
#os.environ['PYSPARK_SUBMIT_ARGS']='C:\spark\spark-2.3.0-bin-hadoop2.7\jars\spark-streaming-kafka-0-8-assembly_2.11-2.3.0-preview.jar'
#os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.spark:spark-sql-kafka-0-10_{SCALA_VERSION}:{SPARK_VERSION} pyspark-shell'
sys.path.insert(0, spark_path + "/bin")
sys.path.insert(0, spark_path + "/python/pyspark/")
sys.path.insert(0, spark_path + "/python/lib/pyspark.zip")
sys.path.insert(0, spark_path + "/python/lib/py4j-0.10.6-src.zip")

## Start Context and Set a checkPoint

In [ ]:
import pyspark
sc = pyspark.SparkContext()
ssc = StreamingContext(sc,60)
#ssc.checkpoint("file:///C:/tmp/xo")
ssc.checkpoint("checkpoint")

## Read From kafka 

In [ ]:
lines = KafkaUtils.createDirectStream(ssc,topics=["rdd1"],kafkaParams={"metadata.broker.list":"localhost:9092"})

## Show stream at screen

In [ ]:
lines.pprint()
ssc.start()
ssc.awaitTermination()

##  count words in each batch without updating state

In [ ]:
#batch is set 1 s 
words =lines.map(lambda x : x[1]).flatMap(lambda x: x.split(" "))
wordcount = words.map(lambda x : (x, 1)).reduceByKey(lambda a,b : a+b)

wordcount.pprint()
ssc.start()

## Count word and update state by Using updateBykey 

In [ ]:

def updateFunction(newValues, runningCount):
    if runningCount is None:
        runningCount = 0
    return sum(newValues, runningCount)

words =lines.map(lambda x : x[1]).flatMap(lambda x: x.split(" "))
wordcount = words.map(lambda x : (x, 1))
runningCounts = wordcount.updateStateByKey(updateFunction)

runningCounts.pprint()
# Start the computation

ssc.start()  